In [ ]:
!pip install  datasets
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_
from datasets import load_dataset
#from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


# compute metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
from datasets import load_dataset
data = load_dataset("surrey-nlp/PLOD-CW")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1072 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/126 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/153 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1072
    })
    validation: Dataset({
        features: ['tokens', 'pos_tags', 'ner_tags'],
        num_rows: 126
    })
    test: Dataset({
        features: ['tokens', 'pos_tags', 'ner_tags'],
        num_rows: 153
    })
})

In [ ]:
# Get the dataset
train_dataset = data["train"]

# Extract the 'ner_tags' feature values
ner_tags_values = train_dataset["ner_tags"]

# Collect unique label names
label_names = list(set(label for tags in ner_tags_values for label in tags))

print(label_names)


['B-AC', 'B-LF', 'I-LF', 'B-O']


In [ ]:
data.set_format(type="pandas")
train_df = data['train'][:]
valid_df = data['validation'][:]
test_df = data['test'][:]

In [ ]:
# this is for training data
# Check if 'text' column already exists
if 'text' not in train_df.columns:
    # Merge 'tokens' and 'pos_tags' into a new column 'text'
    train_df['text'] = train_df.apply(lambda row: ", ".join([f"{token} {pos}" for token, pos in zip(row['tokens'], row['pos_tags'])]), axis=1)

    # Drop the original 'tokens' and 'pos_tags' columns if needed
    train_df = train_df.drop(columns=['tokens', 'pos_tags'])
    print("New 'text' column created.")
else:
    print("The 'text' column already exists in the DataFrame.")

# this is for validation data
if 'text' not in valid_df.columns:
    # Merge 'tokens' and 'pos_tags' into a new column 'text'
    valid_df['text'] = valid_df.apply(lambda row: ", ".join([f"{token} {pos}" for token, pos in zip(row['tokens'], row['pos_tags'])]), axis=1)

    # Drop the original 'tokens' and 'pos_tags' columns if needed
    valid_df = valid_df.drop(columns=['tokens', 'pos_tags'])
    print("New 'text' column created.")
else:
    print("The 'text' column already exists in the DataFrame.")

# this is for test data
if 'text' not in test_df.columns:
    # Merge 'tokens' and 'pos_tags' into a new column 'text'
    test_df['text'] = test_df.apply(lambda row: ", ".join([f"{token} {pos}" for token, pos in zip(row['tokens'], row['pos_tags'])]), axis=1)

    # Drop the original 'tokens' and 'pos_tags' columns if needed
    test_df = test_df.drop(columns=['tokens', 'pos_tags'])
    print("New 'text' column created.")
else:
    print("The 'text' column already exists in the DataFrame.")
train_df[:3]


New 'text' column created.
New 'text' column created.
New 'text' column created.


,ner_tags,text
0,"[B-O, B-O, B-O, B-O, B-LF, I-LF, I-LF, I-LF, I...","For ADP, this DET, purpose NOUN, the DET, Goth..."
1,"[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-LF, I-LF...","The DET, following ADJ, physiological ADJ, tra..."
2,"[B-O, B-AC, B-O, B-O, B-O, B-O, B-O, B-O, B-O,...","Minor ADJ, H PROPN, antigen NOUN, alloimmune A..."


In [ ]:
# Check if the DataFrame has already been processed

# This is for training dataset:
if 'labels' not in train_df.columns or 'text' not in train_df.columns:
    # Split 'ner_tags' and 'text' into separate lists based on commas
    train_df['labels'] = train_df['ner_tags'].apply(lambda x: [tag.split(',')[0].strip() for tag in x])
    train_df['text'] = train_df['text'].apply(lambda x: [tag.strip() for tag in x.split(',')])

    # Explode the lists into separate rows
    train_df = train_df.explode('labels').explode('text')

# This is for Validation dataset:
if 'labels' not in valid_df.columns or 'text' not in valid_df.columns:
    # Split 'ner_tags' and 'text' into separate lists based on commas
    valid_df['labels'] = valid_df['ner_tags'].apply(lambda x: [tag.split(',')[0].strip() for tag in x])
    valid_df['text'] = valid_df['text'].apply(lambda x: [tag.strip() for tag in x.split(',')])

    # Explode the lists into separate rows
    valid_df = valid_df.explode('labels').explode('text')

# This is for Test dataset:
if 'labels' not in test_df.columns or 'text' not in test_df.columns:
    # Split 'ner_tags' and 'text' into separate lists based on commas
    test_df['labels'] = test_df['ner_tags'].apply(lambda x: [tag.split(',')[0].strip() for tag in x])
    test_df['text'] = test_df['text'].apply(lambda x: [tag.strip() for tag in x.split(',')])

    # Explode the lists into separate rows
    test_df = test_df.explode('labels').explode('text')

train_df.tail()

,ner_tags,text,labels
1071,"[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, ...",PCR NOUN,B-O
1071,"[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, ...",experiments NOUN,B-O
1071,"[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, ...",demonstrated VERB,B-O
1071,"[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, ...",that SCONJ,B-O
1071,"[B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, B-O, ...",β PUNCT,B-O


In [ ]:
from sklearn.utils import shuffle

# Define label mapping
label_mapping = {'B-O': 0, 'I-LF': 1, 'B-LF': 2, 'B-AC': 3}

# For training dataset
# Shuffle the DataFrame
train_df = shuffle(train_df, random_state=42)
# Reset the index after shuffling
train_df.reset_index(drop=True, inplace=True)
# Remove the 'ner_tags' column if it exists
if 'ner_tags' in train_df.columns:
    train_df.drop(columns=['ner_tags'], inplace=True)
# Add the 'label' column based on label mapping
train_df['label'] = train_df['labels'].map(label_mapping)

# For Validation dataset
# Shuffle the DataFrame
valid_df = shuffle(valid_df, random_state=42)
# Reset the index after shuffling
valid_df.reset_index(drop=True, inplace=True)
# Remove the 'ner_tags' column if it exists
if 'ner_tags' in valid_df.columns:
    valid_df.drop(columns=['ner_tags'], inplace=True)
# Add the 'label' column based on label mapping
valid_df['label'] = valid_df['labels'].map(label_mapping)

# For testing dataset
# Shuffle the DataFrame
test_df = shuffle(test_df, random_state=42)
# Reset the index after shuffling
test_df.reset_index(drop=True, inplace=True)
# Remove the 'ner_tags' column if it exists
if 'ner_tags' in test_df.columns:
    test_df.drop(columns=['ner_tags'], inplace=True)
# Add the 'label' column based on label mapping
test_df['label'] = test_df['labels'].map(label_mapping)

train_df.head()

,text,labels,label
0,that PRON,B-O,0
1,PUNCT,B-O,0
2,CI NOUN,B-AC,3
3,reduction NOUN,B-O,0
4,mapping NOUN,B-O,0


In [ ]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)


(2196470, 3)
(293025, 3)
(226745, 3)


In [ ]:
print(train_df['labels'].unique())
print(train_df['labels'].value_counts())

['B-O' 'B-AC' 'I-LF' 'B-LF']
labels
B-O     1842151
I-LF     165281
B-AC     117001
B-LF      72037
Name: count, dtype: int64


In [ ]:
# # Calculate the minimum sample size for each label group
# min_sample_size_train = min(len(group) for label, group in train_df.groupby('label'))
# min_sample_size_valid = min(len(group) for label, group in valid_df.groupby('label'))
# min_sample_size_test = min(len(group) for label, group in test_df.groupby('label'))

# # Sample each label group with the minimum sample size
# train_df = train_df.groupby('label').apply(lambda x: x.sample(20000)).reset_index(drop=True)
# valid_df = valid_df.groupby('label').apply(lambda x: x.sample(8000)).reset_index(drop=True)
# test_df = test_df.groupby('label').apply(lambda x: x.sample(3000)).reset_index(drop=True)


In [ ]:
# Sample size for each group
sample_size_train = 20000
sample_size_valid = 8000
sample_size_test = 3000

# Group by 'label' and sample from each group
train_df = train_df.groupby('label').apply(lambda x: x.sample(min(sample_size_train, len(x)))).reset_index(drop=True)
valid_df = valid_df.groupby('label').apply(lambda x: x.sample(min(sample_size_valid, len(x)))).reset_index(drop=True)
test_df = test_df.groupby('label').apply(lambda x: x.sample(min(sample_size_test, len(x)))).reset_index(drop=True)


In [ ]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(80000, 3)
(31571, 3)
(12000, 3)


In [ ]:
print(train_df['label'].value_counts())
print(valid_df['label'].value_counts())
print(test_df['label'].value_counts())


label
0    20000
1    20000
2    20000
3    20000
Name: count, dtype: int64
label
0    8000
1    8000
3    8000
2    7571
Name: count, dtype: int64
label
0    3000
1    3000
2    3000
3    3000
Name: count, dtype: int64


Load Roberta Model and Tokenizer

In [ ]:
from transformers import RobertaTokenizer

PRETRAINED_LM = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
tokenizer


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

RobertaTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [ ]:
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [ ]:
train_input_ids, train_att_masks = encode(train_df['text'].values.tolist())
valid_input_ids, valid_att_masks = encode(valid_df['text'].values.tolist())
test_input_ids, test_att_masks = encode(test_df['text'].values.tolist())

In [ ]:
import torch
train_y = torch.LongTensor(train_df['label'].values.tolist())
valid_y = torch.LongTensor(valid_df['label'].values.tolist())
test_y = torch.LongTensor(test_df['label'].values.tolist())
train_y.size(),valid_y.size(),test_y.size()

(torch.Size([80000]), torch.Size([31571]), torch.Size([12000]))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 32
train_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_dataset = TensorDataset(valid_input_ids, valid_att_masks, valid_y)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
N_labels = len(train_df.label.unique())
N_labels

4

In [ ]:
from transformers import BertForSequenceClassification
N_labels = len(train_df.label.unique())
model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
model = model.cpu()

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

EPOCHS = 3
LEARNING_RATE = 2e-6

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer,
             num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*EPOCHS )

In [ ]:
#collapse-output
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
import numpy as np
import math

train_loss_per_epoch = []
val_loss_per_epoch = []


for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader,desc='Training')):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))


    '''
    Validation
    '''
    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(valid_dataloader,desc='Validation')):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

            loss = output.loss
            valid_loss += loss.item()

            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))

    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)

    '''
    Loss message
    '''
    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(train_df) / BATCH_SIZE), train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(valid_df) / BATCH_SIZE), valid_loss / (step_num_e + 1)))

You can see in the output that the training and validation losses steadily decreases in each epoch.

In [ ]:
from matplotlib import pyplot as plt
epochs = range(1, EPOCHS +1 )
fig, ax = plt.subplots()
ax.plot(epochs,train_loss_per_epoch,label ='training loss')
ax.plot(epochs, val_loss_per_epoch, label = 'validation loss' )
ax.set_title('Training and Validation loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
plt.show()

# Performance Metrics
It's common to use precision, recall, and F1-score as the performance metrics.

In [ ]:
from sklearn.metrics import classification_report
print('classifiation report')
print(classification_report(valid_pred, valid_df['label'].to_numpy(), target_names=label_names))

# Error Analysis
With the predictions, we can plot the confusion matrix:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
def plot_confusion_matrix(y_preds, y_true, labels=None):
  cm = confusion_matrix(y_true, y_preds, normalize="true")
  fig, ax = plt.subplots(figsize=(6, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
  plt.title("Normalized confusion matrix")
  plt.show()

In [ ]:
plot_confusion_matrix(valid_pred,valid_df['label'].to_numpy(),labels=label_names)

# Prediction
Now let's use the trained model to predict the testing set.

In [ ]:
model.eval()
test_pred = []
test_loss= 0
with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        test_loss += loss.item()

        test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
test_pred = np.concatenate(test_pred)

In [ ]:
print('classifiation report')
print(classification_report(test_pred, test_df['label'].to_numpy(),target_names=label_names))

With the predictions, we can plot the confusion matrix again:

In [ ]:
plot_confusion_matrix(test_pred,test_df['label'].to_numpy(),labels=label_names)

Output the misclassified text:

In [ ]:
test_df['pred'] = test_pred
test_df.reset_index(level=0)
print(test_df[test_df['label']!=test_df['pred']].shape)
test_df[test_df['label']!=test_df['pred']][['text','label','pred']].head(10)

In [ ]:
#hide
test_df.to_csv("test_results.csv",index= False)